# Deep Demand Forecasting with Amazon SageMaker

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-2/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

---

**Jupyter Kernel**:
* Please make sure you are using **Python 3 (MXNet 1.9 Python 3.8 CPU Optimized)** kernel

**Run All**: 

* If you are in SageMaker Notebook instance, you can *go to Cell tab -> Run All*
* If you are in SageMaker Studio, you can *go to Run tab -> Run All Cells*

**Overview**

The deep demand forecasting solution contains five sections.

* Stage I: Data preparation and visualization.
* Stage II: Train an optimal LSTNet model using GluonTS with Hyper-Parameter Optimization (HPO).
* Stage III: Train an optimal SageMaker DeepAR model with HPO.
* Stage IV: Evaluate model performance of all three algorithms on the same hold-out test data.

In [ ]:
# Install dependencies files that are used in this notebook.

!pip install altair lunarcalendar plotly
!pip install gluonts==0.8.1
!pip install pystan==2.19.1.1

In [ ]:
import warnings
import json
import sagemaker

warnings.filterwarnings("ignore")
session = sagemaker.Session()

role = sagemaker.get_execution_role()
solution_bucket = "sagemaker-solutions-prod-us-west-2"
solution_name = "Deep-demand-forecasting-with-amazon-sagemaker/2.0.1"
bucket = session.default_bucket()
training_instance_type = "ml.p3.2xlarge"
inference_instance_type = "ml.g4dn.2xlarge"
solution_prefix = "sagemaker-soln-ddf-js--"

## Stage I: Data Preparation and Visualization

### Copy Raw Data to S3

The dataset we use here is the multivariate time-series [electricity consumptions](https://archive.ics.uci.edu/ml/datasets/ElectricityLoadDiagrams20112014) data taken from Dua, D. and Graff, C. (2019). [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/index.php), Irvine, CA: University of California, School of Information and Computer Science. A cleaned version of the data containing **321** time-series with **1 Hour** frequency, starting from **2012-01-01** with **26304** time-steps, is available to download directly via [GluonTS](https://gluon-ts.mxnet.io/). We have also provided the [exchange rate](https://github.com/laiguokun/multivariate-time-series-data/tree/master/exchange_rate) dataset in case you want to try with other datasets as well.

For the ease of access, with have made both of the cleaned datasets available in the following S3 bucket.

**Note:** To reproduce the results from the [blog post](https://towardsdatascience.com/deep-demand-forecasting-with-amazon-sagemaker-e0226410763a) we use `DATASET_NAME="electricity"`.

In [ ]:
DATASET_NAME = "electricity"

In [ ]:
from sagemaker.s3 import S3Downloader

original_bucket = f"s3://{solution_bucket}/0.2.0/{solution_name}"
original_data_prefix = f"artifacts/data/{DATASET_NAME}"
original_data = f"{original_bucket}/{original_data_prefix}"
print("original data: ")
S3Downloader.list(original_data)

First, setup the S3 bucket name and prefix

In [ ]:
prefix = "tst"  # example
raw_data = f"s3://{bucket}/{prefix}"

Copy the `original_data` (s3 bucket in the source account) to our `raw_data` (s3 bucket in your account) if does not exist already

In [ ]:
import boto3

if not S3Downloader.list(raw_data):
    !aws s3 cp --recursive $original_data $raw_data

Download the dependencies wheel files from source S3 to local directory for the usage of `training` and `inference` containers.

In [ ]:
original_dependencies_bucket = f"{original_bucket}/artifacts/dependencies"

# Download the dependencies wheel files from source S3 to local directory for the usage of training and inference containers
!aws s3 cp --recursive --no-progress $original_dependencies_bucket lstnet/dependencies

Set a few variables that are used throughout the notebook

In [ ]:
input_data = raw_data
train_output = f"s3://{bucket}/{prefix}/output"
code_location = f"s3://{bucket}/{prefix}/code"

### Visualize Data

We have provided some utilities in `lstnet/monitor.py` for creating the dataframe from train and test data.

The multivariate time-series electricity consumptions data contains **321** time-series (households) 
with **1 Hour** frequency ranging from year **2012** to **2014**. The training data include hourly electricity consumption values (for the 321 households) 
from **2012-01-01 00:00:00** to **2014-05-26 19:00:00**, and the test data contains values from **2012-01-01 00:00:00** to **2014-06-02 19:00:00** (7 more days of hourly data compared to the training data). 

#### Define **CONTEXT_LENGTH** and **PREDICTION_LENGTH**
To train a time series forecasting model, the **CONTEXT_LENGTH** defines the length of each input time series, and **PREDICTION_LENGTH** defines the length of each output time series.

In [ ]:
CONTEXT_LENGTH = 168  # input length corresponding to 7 days
PREDICTION_LENGTH = 24  # output length corresponding to 1 day


As the `CONTEXT_LENGTH` and `PREDICTION_LENGTH` are set to 168 (7 days) and 24 (next 1 day), we plot the last 7 days of the training data and its subsequent 1 day of the testing data for demonstration purpose. 
The plotted training data and testing data are from **2014-05-19 20:00:00** to **2014-05-26 19:00:00**, and from **2014-05-26 20:00:00** to **2014-05-27 02:00:00**, respectively. 

Prepare the data for visualization.

In [ ]:
!mkdir -p raw_data
!aws s3 cp --recursive $original_data raw_data
%run lstnet/monitor.py

train_df, test_df = prepare_data("raw_data")
NUM_TS = train_df.shape[1] - 1
print(f"raw train data shape {train_df.shape}, test data shape {test_df.shape}")
ts_col_names = [f"ts_{i}" for i in range(NUM_TS + 1)]

In [ ]:
train_df_viz, test_df_viz, selected_cols = create_data_viz(
    train_df, test_df, CONTEXT_LENGTH, PREDICTION_LENGTH, num_sample_ts=11
)

In [ ]:
!pip install typing_extensions==4.4.0
import typing_extensions
from importlib import reload
reload(typing_extensions)

For demonstration purpose, we only plot the first 11 time series out of the 321 time series.

In [ ]:
import altair as alt
from pathlib import Path
from gluonts.dataset.common import MetaData


selection = alt.selection_multi(fields=["covariate"], bind="legend", nearest=True)

train_plot = (
    alt.Chart(train_df_viz, title="Input data")
    .mark_line()
    .encode(
        alt.X("time:T", axis=alt.Axis(title="Time", format="%e %b, %y")),
        alt.Y(
            "value:Q",
            axis=alt.Axis(title="Electricity consumption (kW)"),
            scale=alt.Scale(domain=[0, 1300]),
        ),
        alt.Color("covariate:N"),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .add_selection(selection)
)

meta = MetaData.parse_file(Path("raw_data") / "metadata.json")
timestamps_test = pd.date_range(test_df_viz.iloc[0]["time"], periods=CONTEXT_LENGTH, freq=meta.freq)
ts_start, ts_end = timestamps_test[0], timestamps_test[-1]
test_plot = (
    alt.Chart(test_df_viz, title="Expected output data (ground truth)")
    .mark_line()
    .encode(
        alt.X(
            "time:T",
            axis=alt.Axis(title="Time", format="%e %b, %y"),
            scale=alt.Scale(domain=[ts_start, ts_end]),
        ),
        alt.Y(
            "value:Q",
            axis=alt.Axis(title="Electricity consumption (kW)"),
            scale=alt.Scale(domain=[0, 1300]),
        ),
        alt.Color("covariate:N"),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .add_selection(selection)
)

train_plot | test_plot

## Stage II:  Train an Optimal LSTNet Model using GluonTS with Hyper-Parameter Optimization (HPO)

**LSTNet** is a Deep Learning model that incorporates traditional auto-regressive linear models in parallel to the non-linear neural network part, which makes the non-linear deep learning model more robust for the time series which violate scale changes.

For information on the mathematics behind LSTNet, see [Modeling Long- and Short-Term Temporal Patterns with Deep Neural Networks](https://arxiv.org/abs/1703.07015).

We summarize key actions in this stage as below.

* We firstly train a LSTNet model without HPO. 
* Next, we train an optimal LSTNet model with HPO. 
* We demonstrate that the LSTNet trained with HPO significantly outperform the one trained without HPO.

### Hyperparameters

Here is a set of hyperparameters for LSTNet model.

In [ ]:
hyperparameters = {
    "context_length": CONTEXT_LENGTH,  # sliding window size for training
    "prediction_length": PREDICTION_LENGTH,  # sliding window size for predicting
    "skip_size": 4,  # skip size is used in skip-rnn layer.
    "ar_window": 4,  # auto-regressive window size
    "channels": 72,  # number of convolution channels for the first layer. Note. channels should be divisible by skip_size
    "scaling": True,  # whether to scale the data or not
    "output_activation": "sigmoid",  # output activation function either None, sigmoid or tanh
    "epochs": 15,  # number of epochs for training
    "batch_size": 32,  # number of batch size
    "learning_rate": 1e-2,  # learning rate for weight update
    "dropout_rate": 0.2,  # dropout regularization parameter
    "rnn_cell_type": "gru",  # type of the RNN cell. Either lstm or gru
    "rnn_num_layers": 1,  # number of RNN layers to be used
    "rnn_num_cells": 100,  # number of RNN cells for each layer
    "skip_rnn_cell_type": "gru",  # type of the RNN cell for the skip layer. Either lstm or gru
    "skip_rnn_num_layers": 1,  # number of RNN layers to be used for skip part
    "skip_rnn_num_cells": 10,  # number of RNN cells for each layer for skip part
    "lead_time": 0,  # lead time
    "kernel_size": 6,  # kernel size for first layer Conv2D
}

### Train a LSTNet Model without HPO using SageMaker Estimator

With the hyperparameters defined, we can execute the training job. We use the [GluonTS](https://gluon-ts.mxnet.io/), with **MXNet** as the backend deep learning framework, to define and train our LSTNet model. **Amazon SageMaker** makes it do this with the Framework estimators, which have the deep learning frameworks already setup. Here, we create a SageMaker MXNet estimator and pass in our model training script, hyperparameters, as well as the number and type of training instances we want.

We can then `fit` the estimator on the the `input_data` location in S3. Specifically, the `input_data` contains training and test data, which are in `input_data/train/data.json` and `input_data/test/data.json`, respectively. The training data contains 321 time series from from **2012-01-01 00:00:00** to **2014-05-26 19:00:00**, and the test data contains the same time series from **2012-01-01 00:00:00** to **2014-06-02 19:00:00** (7 more days of hourly data compared to the training data). 


The first 6 days of the test data are further considered as validation data. When HPO is enabled, the optimal hyperparameters are selected based on the validation data. The last 1 day of the test data is considered as the hold-out test set to evaluate the model performance.


Note. When a trained model is evaluated on the test set, only the trailing **PREDICTION_LENGTH** (24) observations of each time series are predicted. The reason to keep entire time series as input (which starts from **2012-01-01 00:00:00**) is to ensure there are enough input observations (i.e., **CONTEXT_LENGTH** observations) for model to make predictions for the trailing **PREDICTION_LENGTH** observations. The evaluation function `evaluate` is used in `lstnet/train.py` and defined in `lstnet/utils.py`.

Training the estimator for 15 epochs takes around **10 minutes**.

In [ ]:
%%time
import time
from sagemaker.mxnet import MXNet
import uuid

!cp lstnet/requirements_train.txt lstnet/requirements.txt
unique_hash = str(uuid.uuid4())[:6]
training_job_name = solution_prefix + unique_hash + "-lstnet-training"
print(
    f"You can go to SageMaker -> Training -> Training jobs -> a job name started with {training_job_name} to monitor training status and details."
)

estimator = MXNet(
    entry_point="train.py",
    source_dir="lstnet",
    role=role,
    instance_count=1,
    instance_type=training_instance_type,  # or "ml.m5.4xlarge" without GPU
    framework_version="1.8.0",
    py_version="py37",
    hyperparameters=hyperparameters,
    base_job_name=training_job_name,
    output_path=train_output,
    code_location=code_location,
    sagemaker_session=session,
    enable_network_isolation=True,  # Set enable_network_isolation=True to ensure a security running environment
    # container_log_level=logging.DEBUG,  # display debug logs
    env={"MMS_DEFAULT_RESPONSE_TIMEOUT": "1000"},
)

start_time = time.time()

estimator.fit(input_data, logs=False)

single_training_job_time_duration = time.time() - start_time  # unit: minute

### Examine the Training Evaluation

We can now access the training artifacts from the specified `output_path` in the above estimator and visualize the training results. 

The current training output are saved in `output.tar.gz` format and donwloaded into notebook for visulization.

In [ ]:
import os

output_path = os.path.join(train_output, estimator._current_job_name, "output")

S3Downloader.download(output_path, "output")
!tar -xvf output/output.tar.gz -C output/

In [ ]:
import pandas as pd

item_metrics = pd.read_csv("output/item_metrics.csv.gz", compression="gzip")
item_metrics.head()

### Visualizing the Training Evaluation

For the visualization we use [altair package](https://github.com/altair-viz/altair) with declarative API. If you want to export to different file formats, follow [altair_saver](https://github.com/altair-viz/altair_saver). 

Note that after exporting to `html` you can go to `output` and open the generated `html` files inside notebook.

Here, we compare the [**Mean Absolute Scaled Error (MASE)**](https://en.wikipedia.org/wiki/Mean_absolute_scaled_error) against the [**symmetric Mean Absolute Percentage Error (sMAPE)**](https://en.wikipedia.org/wiki/Symmetric_mean_absolute_percentage_error). For both metrics, smaller values indicate bettter performance. Thus, the closer the scatter points come to the lower left corner in the plot below, the more accurate the predictions are.

In [ ]:
col_a = "MASE"
col_b = "sMAPE"

alt.data_transformers.disable_max_rows()

scatter = (
    alt.Chart(item_metrics)
    .mark_circle(size=100, fillOpacity=0.8)
    .encode(
        alt.X(col_a, scale=alt.Scale(domain=[-0.5, 10])),
        alt.Y(col_b, scale=alt.Scale(domain=[0, 2.5])),
        tooltip=[col_a, col_b],
    )
    .interactive()
)
scatter.save(os.path.join("output", f"{col_a}_vs_{col_b}.html"))
scatter

In [ ]:
col_a_plot = (
    alt.Chart(item_metrics)
    .mark_bar()
    .encode(
        alt.X(col_a, bin=True),
        y="count()",
    )
)
col_b_plot = (
    alt.Chart(item_metrics)
    .mark_bar()
    .encode(
        alt.X(col_b, bin=True),
        y="count()",
    )
)

col_a_b_plot = col_a_plot | col_b_plot
col_a_b_plot.save(os.path.join("output", f"{col_a}_{col_b}_barplots.html"))
col_a_b_plot

### Deploy an Endpoint

To serve the model, we can deploy an endpoint (which takes around **7 minutes**) where the `lstnet/inference.py` script handles the predictions using the trained model as follows.

In [ ]:
%%time
from sagemaker.mxnet import MXNetModel

!cp lstnet/requirements_inference.txt lstnet/requirements.txt

endpoint_name = solution_prefix + unique_hash + "-lstnet-endpoint"
print(
    f"You can go to SageMaker -> Inference -> Endpoints --> an endpoint with name {endpoint_name} to monitor the deployment status."
)

model = MXNetModel(
    model_data=os.path.join(output_path, "model.tar.gz"),
    role=role,
    entry_point="inference.py",
    source_dir="lstnet",
    py_version="py37",
    name=solution_prefix + "-model",
    framework_version="1.8.0",
    code_location=code_location,
    enable_network_isolation=True,  # Set enable_network_isolation=True to ensure a security running environment
    env={"MMS_DEFAULT_RESPONSE_TIMEOUT": "1000"},
)

predictor = model.deploy(
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
    initial_instance_count=1,
)

### Testing the Endpoint

To do sanity checking, here we can test the endpoint by requesting predictions for a randomly generated data. The `predictor` handles serialization and deserialization of the requests.

In [ ]:
train_df, test_df = prepare_data("raw_data")
NUM_TS = train_df.shape[1] - 1

In [ ]:
%%time
import numpy as np

np.random.seed(1)
random_test = np.random.randn(NUM_TS, PREDICTION_LENGTH)

# json serializable request format
random_test_data = {}
random_test_data["target"] = random_test.tolist()
random_test_data["start"] = "2014-01-01"
random_test_data["source"] = []

random_ret = predictor.predict(random_test_data)

Load the returned JSON objects into numpy array.

In [ ]:
forecasts = np.array(random_ret["forecasts"]["samples"])
print(f"Forecasts shape with 10 samples: {forecasts.shape}")

### Query the Endpoint for Prediction and Compute the Evaluation Metrics
Here we prepare our test data for prediction. Since we have trained our model given the hyperparameters defined earlier `CONTEXT_LENGTH` (length of input time series) and `PREDICTION_LENGTH` (length of output time series), 
we can now input the final window in the test data (a hold-out data without being used in the training and validation) to our model to test from **2014-06-01 20:00:00** onwards, get the predictions, and compute the evaluation metrics.

In [ ]:
%%time

start_time = time.time()
test_data = {}
test_data["target"] = (
    test_df.iloc[-(CONTEXT_LENGTH + PREDICTION_LENGTH) :].set_index("time").values.T.tolist()
)
test_data["start"] = "2014-05-25 20:00:00"
test_data["source"] = []

predictions = predictor.predict(test_data)
inference_time_duration = time.time() - start_time

Summarize the evaluation metrics on the test data

In [ ]:
dict_agg_metrics_without_hpo = json.loads(predictions["agg_metrics"])
dict_agg_metrics_without_hpo["RRSE"] = dict_agg_metrics_without_hpo["RMSE"] / np.std(
    test_data["target"]
)  # compute Root Relative Squared Error (RRSE)
dict_agg_metrics_without_hpo["Training Time (min)"] = single_training_job_time_duration / 60
dict_agg_metrics_without_hpo["Inference Time (s)"] = inference_time_duration
df_agg_metrics_without_hpo = pd.DataFrame.from_dict(dict_agg_metrics_without_hpo, orient="index")

### Compute the Evaluation Metrics
Except for the training and inference time, for RRSE (Root Relative Squared Error), MAPE (Mean Absolute Percentage Error), and sMAPE (symmetric Mean Absolute Percentage Error), smaller values indicate better predictive performance.

In [ ]:
METRICS_TO_SHOW = ["RRSE", "MAPE", "sMAPE", "Training Time (min)", "Inference Time (s)"]
print("Evaluation metrics on the test data are shown as below.")
df_agg_metrics_without_hpo_eval_metrics = df_agg_metrics_without_hpo.reindex(METRICS_TO_SHOW)
df_agg_metrics_without_hpo_eval_metrics.columns = ["LSTNet without HPO"]
print("For the evaluation metrics, smaller value indicates better predictive performance.")
df_agg_metrics_without_hpo_eval_metrics

### Interactive Visualization

It is important to visualize how the model is performing given the test data. Based on the predictions from **2014-06-01 20:00:00** onwards by querying the endpoint shown above, 
we visualize the performance of the model for a sample of time-series.

In [ ]:
print(f"prepared train data shape {train_df.shape}, test data shape {test_df.shape}")
ts_col_names = [f"ts_{i}" for i in range(NUM_TS + 1)]
train_df_viz, test_df_viz, selected_cols = create_data_viz(
    test_df.iloc[: test_df.shape[0] - PREDICTION_LENGTH],
    test_df,
    CONTEXT_LENGTH,
    PREDICTION_LENGTH,
    num_sample_ts=test_df.shape[1] - 1,
)
train_df_viz.head()

In [ ]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

forecasts = np.transpose(np.array(predictions["forecasts"]["samples"][0]))

preds = ListDataset(
    [
        {
            FieldName.TARGET: forecasts,
            FieldName.START: predictions["forecasts"]["start_date"],
        }
    ],
    freq=predictions["forecasts"]["freq"],
    one_dim_target=False,
)

preds_df = multivar_df(next(iter(preds)))
preds_df_filter = preds_df.loc[:, ["time"] + selected_cols]
preds_df_filter = pd.melt(preds_df_filter, id_vars=["time"], value_vars=selected_cols)
preds_df_filter.rename(columns={"variable": "covariate"}, inplace=True)
preds_df_filter.head()

For demonstration purpose, we randomly select 10 time series to visualize model predictions to compare with the ground truth data.

In [ ]:
selected_ts = [
    "ts_98",
    "ts_100",
    "ts_116",
    "ts_137",
    "ts_147",
    "ts_156",
    "ts_184",
    "ts_216",
    "ts_245",
    "ts_267",
]

In [ ]:
selection = alt.selection_multi(fields=["covariate"], bind="legend", nearest=True)

train_plot = (
    alt.Chart(
        train_df_viz.loc[train_df_viz["covariate"].isin(selected_ts)],
        title="Input data",
    )
    .mark_line()
    .encode(
        alt.X("time:T", axis=alt.Axis(title="Time", format="%e %b, %y")),
        alt.Y(
            "value:Q",
            axis=alt.Axis(title="Electricity consumption (kW)"),
            scale=alt.Scale(domain=[0, 3000]),
        ),
        alt.Color("covariate:N"),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .add_selection(selection)
)

timestamps_test = pd.date_range(test_df_viz.iloc[0]["time"], periods=CONTEXT_LENGTH, freq=meta.freq)
ts_start, ts_end = timestamps_test[0], timestamps_test[-1]
test_plot = (
    alt.Chart(
        test_df_viz.loc[test_df_viz["covariate"].isin(selected_ts)],
        title="Expected output data (ground truth)",
    )
    .mark_line()
    .encode(
        alt.X(
            "time:T",
            axis=alt.Axis(title="Time", format="%e %b, %y"),
            scale=alt.Scale(domain=[ts_start, ts_end]),
        ),
        alt.Y(
            "value:Q",
            axis=alt.Axis(title="Electricity consumption (kW)"),
            scale=alt.Scale(domain=[0, 3000]),
        ),
        alt.Color("covariate:N"),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .add_selection(selection)
)

preds_plot = (
    alt.Chart(
        preds_df_filter.loc[preds_df_filter["covariate"].isin(selected_ts)],
        title="Model predictions",
    )
    .mark_line()
    .encode(
        alt.X(
            "time:T",
            axis=alt.Axis(title="Time", format="%e %b, %y"),
            scale=alt.Scale(domain=[ts_start, ts_end]),
        ),
        alt.Y(
            "value:Q",
            axis=alt.Axis(title="Electricity consumption (kW)"),
            scale=alt.Scale(domain=[0, 3000]),
        ),
        alt.Color("covariate:N"),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .add_selection(selection)
)

(train_plot | test_plot) & preds_plot

Note. The model predictions shown in above plots are not optimal as the HPO is not enabled. In next section, we train an optimal LSTNet model using HPO. The visualizations of model predictions from the optimal LSTNet are shown in the Stage V with those from all the other models.

### Train an Optimal LSTNet Model with HPO using SageMaker Estimator

In this section we further improve the model performance by adding HPO tuning with [SageMaker Automatic Model Tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html). 
Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose.
The best model and its corresponded hyperparmeters are selected on the validation data from **2014-05-26 20:00:00** to **2014-06-01 19:00:00** (corresponding to 6 days). Next, the best model is evaluated on the hold-out test data from 
**2014-06-01 20:00:00** to **2014-06-02 19:00:00** (corresponding the next 1 day). Finally, we show that the performance of model trained with HPO is significantly better than the one trained without HPO.


This package depends on and incorporates or retrieves a third-party software package (such as an open source package) at install-time or build-time or run-time ("External Dependency"). The External Dependency is subject to a license term 
that you must accept in order to use this package. If you do not
accept the applicable license term, you should not use this package. We
recommend that you consult your company’s open source approval policy before
proceeding.

Provided below is the External Dependency and the applicable license
identification as indicated by the documentation associated with the External
Dependency as of Amazon's most recent review.

Package: PyMeeus. License: [GNU Lesser General Public License v3 (LGPLv3) (LGPLv3)](https://github.com/architest/pymeeus/blob/master/LICENSE.txt).


In [ ]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

# Static hyperparameters we do not tune
hyperparameters = {
    "context_length": CONTEXT_LENGTH,  # sliding window size for training
    "prediction_length": PREDICTION_LENGTH,  # sliding window size for predicting
    "skip_size": 4,  # skip size is used in skip-rnn layer
    "ar_window": 4,  # auto-regressive window size
    "scaling": True,  # whether to scale the data or not
    "batch_size": 32,  # number of batch size
    "rnn_cell_type": "gru",  # type of the RNN cell. Either lstm or gru (default: gru)
    "rnn_num_layers": 1,  # number of RNN layers to be used
    "rnn_num_cells": 100,  # number of RNN cells for each layer
    "skip_rnn_cell_type": "gru",  #  type of the RNN cell for the skip layer. Either lstm or gru
    "skip_rnn_num_layers": 1,  # number of RNN layers to be used for skip part
    "skip_rnn_num_cells": 10,  # number of RNN cells for each layer for skip part
    "lead_time": 0,  # lead time
    "kernel_size": 6,  # kernel size for first layer Conv2D
}

# Dynamic hyperparameters we want to tune and their searching ranges. For demonstartion purpose, we skip the architecture search by skipping tunning the hyperparameters such as 'skip_rnn_num_layers', 'rnn_num_layers', and etc.
hyperparameter_ranges = {
    "dropout_rate": ContinuousParameter(0.3, 0.5),
    "channels": CategoricalParameter(
        [80, 84, 88, 92, 96]
    ),  # Note. channels should be divisible by skip_size
    "output_activation": CategoricalParameter(["sigmoid", "tanh"]),
    "learning_rate": ContinuousParameter(0.001, 0.01),
    "epochs": IntegerParameter(40, 50),
}

Define the objective metric name, metric definiton (with regex pattern), and objective type for the tuning job.

In [ ]:
objective_metric_name = "Root Relative Squared Error (RRSE)"
metric_definitions = [
    {
        "Name": "Root Relative Squared Error (RRSE)",
        "Regex": "Root Relative Squared Error \\(RRSE\\): (\\S+)",
    }
]  # Root Relative Squared Error (RSE):
objective_type = "Minimize"

In [ ]:
%%time
!cp lstnet/requirements_train.txt lstnet/requirements.txt

estimator_tuning = MXNet(
    entry_point="train.py",
    source_dir="lstnet",
    role=role,
    instance_count=1,
    instance_type=training_instance_type,  # we use CPU machine here as there are multiple training jobs being triggered. You can also change it back to GPU machine ml.g4dn.2xlarge
    framework_version="1.8.0",
    py_version="py37",
    hyperparameters=hyperparameters,
    output_path=train_output,
    code_location=code_location,
    sagemaker_session=session,
    enable_network_isolation=True,  # Set enable_network_isolation=True to ensure a security running environment
    # container_log_level=logging.DEBUG,  # display debug logs
    env={"MMS_DEFAULT_RESPONSE_TIMEOUT": "1000"},
)

In [ ]:
tuning_job_name = solution_prefix + unique_hash + "-lstnet-tuning"
print(
    f"You can go to SageMaker -> Training -> Hyperparameter tuning jobs -> a job name started with {tuning_job_name} to monitor HPO tuning status and details.\n"
    f"Note. You are not able to successfully run the following cells until the tuning job completes. This step may take around 1 hour."
)

tuner = HyperparameterTuner(
    estimator_tuning,  # using the estimator defined in Stage I
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=20,
    max_parallel_jobs=3,
    objective_type=objective_type,
    base_tuning_job_name=tuning_job_name,
)

start_time = time.time()

tuner.fit(input_data)

hpo_training_job_time_duration = time.time() - start_time

Find the tuning job name

In [ ]:
sm_client = boto3.Session().client("sagemaker")

tuning_job_name = tuner.latest_tuning_job.name
tuning_job_name

Checking the status of the tuning jobs: whether all of them are finished with 'Completed' status.

In [ ]:
tuning_job_result = sm_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

is_minimize = (
    tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["Type"]
    != "Minimize"
)
objective_name = tuning_job_result["HyperParameterTuningJobConfig"][
    "HyperParameterTuningJobObjective"
]["MetricName"]

Once the tuning job finishes, we can bring in a table of metrics.

In [ ]:
tuner_analytics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner_analytics.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=True)
        print("Number of training jobs with valid objective: %d" % len(df))
        print(
            {
                "lowest": min(df["FinalObjectiveValue"]),
                "highest": max(df["FinalObjectiveValue"]),
            }
        )
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Get the output path for the best model from the HPO tuning.

In [ ]:
df = df[df["TrainingJobStatus"] == "Completed"]  # filter out the failed jobs
output_path_best_tuning_job = os.path.join(train_output, df["TrainingJobName"].iloc[0], "output")

In [ ]:
print(f"The output path of the best model from the hpo tuning is: {output_path_best_tuning_job}")

In [ ]:
%%time
!cp lstnet/requirements_inference.txt lstnet/requirements.txt

endpoint_name = solution_prefix + unique_hash + "-lstnet-tuning-endpoint"
print(
    f"You can go to SageMaker -> Inference -> Endpoints --> an endpoint with name {endpoint_name} to monitor the deployment status."
)

tuning_best_model = MXNetModel(
    model_data=os.path.join(output_path_best_tuning_job, "model.tar.gz"),
    role=role,
    entry_point="inference.py",
    source_dir="lstnet",
    py_version="py37",
    name=solution_prefix + "-hpo-tuning-model",
    framework_version="1.8.0",
    code_location=code_location,
    enable_network_isolation=True,  # Set enable_network_isolation=True to ensure a security running environment
    env={"MMS_DEFAULT_RESPONSE_TIMEOUT": "1000"},
)

tuning_predictor = tuning_best_model.deploy(
    instance_type=inference_instance_type,
    endpoint_name=endpoint_name,
    initial_instance_count=1,
)

After deploying the endpoint, we query the endpoint using the same test data as defined in training LSTNet without HPO, compute the evaluaton metrics, and compare with the results without HPO tuning.

In [ ]:
start_time = time.time()
predictions_tuning = tuning_predictor.predict(test_data)
predicted_lstnet = predictions_tuning["forecasts"]["samples"]
inference_time_duration_tuning = time.time() - start_time

In [ ]:
dict_agg_metrics_with_hpo = json.loads(predictions_tuning["agg_metrics"])
dict_agg_metrics_with_hpo["RRSE"] = dict_agg_metrics_with_hpo["RMSE"] / np.std(
    test_data["target"]
)  # compute Root Relative Squared Error (RRSE)
dict_agg_metrics_with_hpo["Training Time (min)"] = hpo_training_job_time_duration / 60
dict_agg_metrics_with_hpo["Inference Time (s)"] = inference_time_duration_tuning
df_agg_metrics_with_hpo = pd.DataFrame.from_dict(dict_agg_metrics_with_hpo, orient="index")

In [ ]:
# Combine with the evaluation metrics without hpo tuning
df_agg_metrics_with_hpo_eval_metrics = df_agg_metrics_with_hpo.reindex(METRICS_TO_SHOW)
df_agg_metrics_with_hpo_eval_metrics.columns = ["LSTNet with HPO"]
df_agg_metrics_without_hpo_eval_metrics.join(df_agg_metrics_with_hpo_eval_metrics)

We can see the metrics values with HPO tuning is smaller than those without HPO tuning on the same test data. This indicates that HPO tuning further improves the model performance. 

## Stage III: Train an Optimal SageMaker DeepAR Model with HPO.

The [Amazon SageMaker DeepAR](https://docs.aws.amazon.com/sagemaker/latest/dg/deepar.html) forecasting algorithm is a supervised learning algorithm for forecasting scalar (one-dimensional) time series using recurrent neural networks (RNN). Classical forecasting methods, such as autoregressive integrated moving average (ARIMA) or exponential smoothing (ETS), fit a single model to each individual time series. They then use that model to extrapolate the time series into the future.

In many applications, however, you have many similar time series across a set of cross-sectional units. For example, you might have time series groupings for demand for different products, server loads, and requests for webpages. For this type of application, you can benefit from training a single model jointly over all of the time series. DeepAR takes this approach. When your dataset contains hundreds of related time series, DeepAR outperforms the standard ARIMA and ETS methods. You can also use the trained model to generate forecasts for new time series that are similar to the ones it has been trained on.

For information on the mathematics behind DeepAR, see [DeepAR: Probabilistic Forecasting with Autoregressive Recurrent Networks](https://arxiv.org/abs/1704.04110).

Similar to the setting in the previous stages, we do following:

* We firstly train a DeepAR model without HPO. 
* Next, we train an optimal DeepAR model with HPO. 
* We demonstrate that the DeepAR trained with HPO significantly outperform the one trained without HPO.

### Prepare the traing, validation, and test data

In [ ]:
from typing import Tuple, Optional, List
from datetime import timedelta
from gluonts.dataset.rolling_dataset import StepStrategy, generate_rolling_dataset
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.common import TrainDatasets, load_datasets

Read the data from local directory `raw_data`

In [ ]:
path = Path("raw_data")
metadata_path = path if path == Path("raw_data") else path / "metadata"
ds = load_datasets(metadata_path, path / "train", path / "test")
target_dim = int(ds.metadata.feat_static_cat[0].cardinality)

In [ ]:
freq = ds.metadata.freq
ts_first_dim = next(iter(ds.train))
start_dataset = ts_first_dim["start"]
ts_train_len = ts_first_dim["target"].shape[0]
end_dataset = pd.date_range(start_dataset, periods=ts_train_len, freq=freq)[-1]

Reformat the training data as input for DeepAR.

In [ ]:
training_data = [
    {
        "start": str(start_dataset),
        "target": ts[
            "target"
        ].tolist(),  # We use -1, because pandas indexing includes the upper bound
    }
    for ts in ds.train
]
print(f"Train data contains {len(training_data)} time series.")

Since the first 6 days of test data are used as the valiation set (same setting as in the previous evaluation), we generate validation data that extends to 1, 2, 3, 4, 5, 6 days beyond the training range. This way we perform rolling evaluation of our model.

In [ ]:
num_valid_windows = 6

validation_data = [
    {
        "start": str(ts["start"]),
        "target": ts["target"][: (ts_train_len + k * PREDICTION_LENGTH)].tolist(),
    }
    for k in range(1, num_valid_windows + 1)
    for ts in list(ds.test)[-target_dim:]
]
print(f"The validation data contains {len(validation_data)} time series.")

In [ ]:
def write_dicts_to_file(path, data):
    with open(path, "wb") as fp:
        for d in data:
            fp.write(json.dumps(d).encode("utf-8"))
            fp.write("\n".encode("utf-8"))

Write the train and validation file into local directory `input_data_deepar`

In [ ]:
!mkdir -p input_data_deepar

write_dicts_to_file("input_data_deepar/train.json", training_data)
write_dicts_to_file("input_data_deepar/validation.json", validation_data)

Then we copy them to the S3 bucket where the DeepAR reads the input data from.

In [ ]:
s3 = boto3.resource("s3")


def copy_to_s3(local_file, s3_path, override=True):
    assert s3_path.startswith("s3://")
    split = s3_path.split("/")
    bucket = split[2]
    path = "/".join(split[3:])
    buk = s3.Bucket(bucket)

    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print(
                "File s3://{}/{} already exists.\nSet override to upload anyway.\n".format(
                    s3_bucket, s3_path
                )
            )
            return
        else:
            print("Overwriting existing file")
    with open(local_file, "rb") as data:
        print("Uploading file to {}".format(s3_path))
        buk.put_object(Key=path, Body=data)

In [ ]:
s3_input_path_deepar = f"s3://{bucket}/{prefix}/input_data_deepar"
s3_output_path_deepar = f"s3://{bucket}/{prefix}/output_deepar"

copy_to_s3("input_data_deepar/train.json", s3_input_path_deepar + "/train/train.json")
copy_to_s3(
    "input_data_deepar/validation.json",
    s3_input_path_deepar + "/validation/validation.json",
)

### Train a SageMaker DeepAR Model without HPO


In [ ]:
import boto3

boto3_session = boto3.session.Session()
region = boto3_session.region_name
image_name = sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest")

In [ ]:
training_job_name_deepar = solution_prefix + unique_hash + "-deepar-training"

deepar_estimator = sagemaker.estimator.Estimator(
    image_uri=image_name,
    sagemaker_session=session,
    role=role,
    train_instance_count=1,  # change it to 2 for distributed training to accelerate training
    train_instance_type=training_instance_type,  # we use a GPU machine here to shorten HPO runtime as there are multiple training jobs being triggered. You can also change it to a CPU machine 'ml.m5.4xlarge'.
    base_job_name=training_job_name_deepar,
    output_path=s3_output_path_deepar,
)

In [ ]:
hyperparameters = {
    "time_freq": freq,
    "epochs": "400",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(CONTEXT_LENGTH),
    "prediction_length": str(PREDICTION_LENGTH),
}

In [ ]:
deepar_estimator.set_hyperparameters(**hyperparameters)

In [ ]:
data_channels = {
    "train": "{}/train/".format(s3_input_path_deepar),
    "test": "{}/validation/".format(s3_input_path_deepar),
}

print(
    f"You can go to SageMaker -> Training -> Training jobs -> a job name started with {training_job_name_deepar} to monitor training status and details."
)

start_time = time.time()
deepar_estimator.fit(inputs=data_channels, wait=True, logs=False)
deepar_training_job_time_duration = time.time() - start_time

### Deploy the Endpoint

In [ ]:
from sagemaker.serializers import IdentitySerializer
from sagemaker.deserializers import JSONDeserializer

Following utility class allows for using `pandas.Series` objects to query the endpoint.

In [ ]:
def series_to_obj(ts, cat=None):
    obj = {"start": str(ts.index[0]), "target": list(ts)}
    if cat:
        obj["cat"] = cat
    return obj


class DeepARPredictor(sagemaker.predictor.Predictor):
    def __init__(self, *args, **kwargs):
        super().__init__(
            *args,
            serializer=IdentitySerializer(content_type="application/json"),
            **kwargs,
        )

    def set_prediction_parameters(self, freq, prediction_length):
        """Set the time frequency and prediction length parameters. This method **must** be
        called before being able to use `predict`.

        Parameters:
        freq -- string indicating the time frequency
        prediction_length -- integer, number of predicted time points

        Return value: none.
        """
        self.freq = freq
        self.prediction_length = prediction_length

    def predict(
        self,
        ts,
        cat=None,
        encoding="utf-8",
        num_samples=100,
        quantiles=["0.1", "0.5", "0.9"],
    ):
        """Requests the prediction of for the time series listed in `ts`, each with the
        (optional) corresponding category listed in `cat`.

        Parameters:
        ts -- list of `pandas.Series` objects, the time series to predict
        cat -- list of integers (default: None)
        encoding -- string, encoding to use for the request (default: 'utf-8')
        num_samples -- integer, number of samples to compute at prediction time (default: 100)
        quantiles -- list of strings specifying the quantiles to compute (default: ['0.1', '0.5', '0.9'])

        Return value: list of `pandas.DataFrame` objects, each containing the predictions
        """
        prediction_times = [x.index[-1] + x.index.freq for x in ts]
        req = self.__encode_request(ts, cat, encoding, num_samples, quantiles)
        res = super(DeepARPredictor, self).predict(req)
        return self.__decode_response(res, prediction_times, encoding)

    def __encode_request(self, ts, cat, encoding, num_samples, quantiles):
        instances = [series_to_obj(ts[k], cat[k] if cat else None) for k in range(len(ts))]
        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles", "mean"],
            "quantiles": quantiles,
        }
        http_request_data = {"instances": instances, "configuration": configuration}
        return json.dumps(http_request_data).encode(encoding)

    def __decode_response(self, response, prediction_times, encoding):
        response_data = json.loads(response.decode(encoding))
        list_of_df = []
        for k in range(len(prediction_times)):
            prediction_index = pd.date_range(
                start=prediction_times[k],
                freq=self.freq,
                periods=self.prediction_length,
            )
            res_df = pd.DataFrame(
                data=response_data["predictions"][k]["quantiles"],
                index=prediction_index,
            )
            sample_mean_predict_df = pd.DataFrame(
                {
                    "mean": response_data["predictions"][k]["mean"],
                },
                index=prediction_index,
            )
            res_df = pd.concat([res_df, sample_mean_predict_df], axis=1)
            list_of_df.append(res_df)
        return list_of_df

Different from GluonTS algorithms, the test data should not contain the values you want to predict.

In [ ]:
test_data = [
    pd.Series(
        ts["target"][:-PREDICTION_LENGTH].tolist(),
        index=pd.date_range(
            start=ts["start"],
            periods=ts["target"][:-PREDICTION_LENGTH].shape[0],
            freq=freq,
        ),
    )
    for ts in list(ds.test)[-target_dim:]
]
print(len(test_data))

In [ ]:
endpoint_name_deepar = solution_prefix + unique_hash + "-deepar-endpoint"
print(
    f"You can go to SageMaker -> Inference -> Endpoints --> an endpoint with name {endpoint_name_deepar} to monitor the deployment status."
)

deepar_endpoint = deepar_estimator.deploy(
    initial_instance_count=1,
    endpoint_name=endpoint_name_deepar,
    instance_type=inference_instance_type,
    serializer=IdentitySerializer(content_type="application/json"),
    deserializer=JSONDeserializer(),
)

In [ ]:
deepar_predictor = DeepARPredictor(
    endpoint_name=deepar_endpoint.endpoint_name, sagemaker_session=session
)

After the endpoint is deployed, we query the endpoint to get model predictions for the test data.

In [ ]:
start_time = time.time()

deepar_predictor.set_prediction_parameters(freq, PREDICTION_LENGTH)
predictions = []
for i in range(len(test_data)):
    predictions.extend(deepar_predictor.predict(test_data[i : (i + 1)], num_samples=100))

deepar_inference_time_duration = time.time() - start_time

In [ ]:
forecasts_all = []
forecasts_all.extend(
    [
        SampleForecast(
            samples=np.expand_dims(np.array(x["mean"]), axis=0),
            start_date=x["mean"].index[0],
            freq=freq,
        )
        for x in predictions
    ]
)

Compute the evaluation metrics.

In [ ]:
evaluator = Evaluator()
tss_all = [
    pd.Series(
        ts["target"].tolist(),
        index=pd.date_range(start=ts["start"], periods=ts["target"].shape[0], freq=freq),
    )
    for ts in list(ds.test)[-target_dim:]
]

In [ ]:
agg_metrics, item_metrics = evaluator(iter(tss_all), iter(forecasts_all), num_series=len(test_data))

In [ ]:
deepar_dict_agg_metrics_without_hpo = agg_metrics
deepar_dict_agg_metrics_without_hpo["RRSE"] = deepar_dict_agg_metrics_without_hpo["RMSE"] / np.std(
    tss_all
)  # compute Root Relative Squared Error (RRSE)
deepar_dict_agg_metrics_without_hpo["Training Time (min)"] = deepar_training_job_time_duration / 60
deepar_dict_agg_metrics_without_hpo["Inference Time (s)"] = deepar_inference_time_duration
deepar_dict_agg_metrics_without_hpo = pd.DataFrame.from_dict(
    deepar_dict_agg_metrics_without_hpo, orient="index", columns=["DeepAR without HPO"]
)

In [ ]:
deepar_dict_agg_metrics_without_hpo = deepar_dict_agg_metrics_without_hpo.reindex(METRICS_TO_SHOW)
deepar_dict_agg_metrics_without_hpo

### Train an Optimal SageMaker DeepAR with HPO

In [ ]:
deepar = sagemaker.estimator.Estimator(
    image_uri=image_name,
    sagemaker_session=session,
    role=role,
    train_instance_count=1,
    train_instance_type=training_instance_type,  # we use a GPU machine here to shorten HPO runtime as there are multiple training jobs being triggered. You can also change it to a CPU machine 'ml.m5.2xlarge'.
    output_path=s3_output_path_deepar,
)

In [ ]:
hyperparameters = {
    "time_freq": freq,
    "early_stopping_patience": "40",
    "context_length": str(CONTEXT_LENGTH),
    "prediction_length": str(PREDICTION_LENGTH),
}

deepar.set_hyperparameters(**hyperparameters)

In [ ]:
hyperparameter_ranges = {
    "mini_batch_size": IntegerParameter(100, 400),
    "epochs": IntegerParameter(200, 400),
    "num_cells": IntegerParameter(50, 100),
    "likelihood": CategoricalParameter(["negative-binomial", "student-T"]),
    "learning_rate": ContinuousParameter(0.001, 0.1),
}

Note. To reduce HPO runtime, please adjust parameter `max_parallel_jobs` in function `HyperparameterTuner` below, depending on number of available instances in your account. Current `max_parallel_jobs` is set as 3.

In [ ]:
objective_metric_name = "test:RMSE"  # RMSE * (standard deviation of training data) =  RRSE, since the standard deviation of training data is fixed. Optimizing RMSE is the same as optimizing RRSE.

tuning_job_name_deepar = solution_prefix + unique_hash + "-deepar-tuning"
print(
    f"You can go to SageMaker -> Training -> Hyperparameter tuning jobs -> a job name started with {tuning_job_name_deepar} to monitor HPO tuning status and details.\n"
    f"Note. You are not able to successfully run the following cells until the tuning job completes. This step may take up to 2-3 hours depending on parameter max_parallel_jobs."
)

start_time = time.time()
deepar_tuner = HyperparameterTuner(
    deepar,
    objective_metric_name,
    hyperparameter_ranges,
    max_jobs=20,
    strategy="Bayesian",
    objective_type="Minimize",
    max_parallel_jobs=3,
    early_stopping_type="Auto",
    base_tuning_job_name=tuning_job_name_deepar,
)

deepar_tuner.fit(data_channels, include_cls_metadata=False)

deepar_training_job_time_duration_hpo = time.time() - start_time

### Examine the HPO tuning job results

Find the tuning job name

In [ ]:
sm_client = boto3.Session().client("sagemaker")

tuning_job_name = deepar_tuner.latest_tuning_job.name
tuning_job_name

Checking the status of the tuning jobs: whether all of them are finished with 'Completed' status.

In [ ]:
tuning_job_result = sm_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

is_minimize = (
    tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["Type"]
    != "Minimize"
)
objective_name = tuning_job_result["HyperParameterTuningJobConfig"][
    "HyperParameterTuningJobObjective"
]["MetricName"]

Once the tuning job finishes, we can bring in a table of metrics.

In [ ]:
tuner_analytics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner_analytics.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=True)
        print("Number of training jobs with valid objective: %d" % len(df))
        print(
            {
                "lowest": min(df["FinalObjectiveValue"]),
                "highest": max(df["FinalObjectiveValue"]),
            }
        )
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

### Deploy an Endpoint for the Best Tuning Job

In [ ]:
endpoint_name_deepar_hpo = solution_prefix + unique_hash + "-deepar-tuning-endpoint"
print(
    f"You can go to SageMaker -> Inference -> Endpoints --> an endpoint with name {endpoint_name_deepar_hpo} to monitor the deployment status."
)

deepar_endpoint_hpo = deepar_tuner.deploy(
    initial_instance_count=1,
    endpoint_name=endpoint_name_deepar_hpo,
    instance_type=inference_instance_type,
    serializer=IdentitySerializer(content_type="application/json"),
    deserializer=JSONDeserializer(),
    wait=True,
)

In [ ]:
deepar_predictor_hpo = DeepARPredictor(
    endpoint_name=deepar_endpoint_hpo.endpoint_name, sagemaker_session=session
)

In [ ]:
start_time = time.time()

deepar_predictor_hpo.set_prediction_parameters(freq, PREDICTION_LENGTH)
predictions_hpo = []
for i in range(len(test_data)):
    predictions_hpo.extend(deepar_predictor_hpo.predict(test_data[i : (i + 1)], num_samples=100))

deepar_inference_time_duration_hpo = time.time() - start_time

In [ ]:
forecasts_all_hpo = []
forecasts_all_hpo.extend(
    [
        SampleForecast(
            samples=np.expand_dims(np.array(x["mean"]), axis=0),
            start_date=x["mean"].index[0],
            freq=freq,
        )
        for x in predictions_hpo
    ]
)
predicted_deepar = forecasts_all_hpo

In [ ]:
agg_metrics_hpo, item_metrics_hpo = evaluator(
    iter(tss_all), iter(forecasts_all_hpo), num_series=len(test_data)
)

In [ ]:
deepar_dict_agg_metrics_with_hpo = agg_metrics_hpo
deepar_dict_agg_metrics_with_hpo["RRSE"] = deepar_dict_agg_metrics_with_hpo["RMSE"] / np.std(
    tss_all
)  # compute Root Relative Squared Error (RRSE)
deepar_dict_agg_metrics_with_hpo["Training Time (min)"] = deepar_training_job_time_duration_hpo / 60
deepar_dict_agg_metrics_with_hpo["Inference Time (s)"] = deepar_inference_time_duration_hpo
deepar_dict_agg_metrics_with_hpo = pd.DataFrame.from_dict(
    deepar_dict_agg_metrics_with_hpo, orient="index", columns=["DeepAR with HPO"]
)

In [ ]:
deepar_results_all = pd.concat(
    [
        deepar_dict_agg_metrics_without_hpo,
        deepar_dict_agg_metrics_with_hpo.reindex(METRICS_TO_SHOW),
    ],
    axis=1,
)
deepar_results_all

We can see the metrics values with HPO tuning is smaller than those without HPO tuning on the same test data. This indicates that HPO tuning further improves the model performance. 

## Stage IV: Evaluate Model Performance of All Three Algorithms on the Same Hold-out Test Data.

### Compare the Model Performance from the Three Models Trained from HPO

Except for the training and inference time, for RRSE (Root Relative Squared Error), MAPE (Mean Absolute Percentage Error), and sMAPE (symmetric Mean Absolute Percentage Error), smaller values indicate better predictive performance.

In [ ]:
df_agg_metrics_with_hpo_eval_metrics.join(
    deepar_dict_agg_metrics_with_hpo
)

### Visualize Model Predictions

In [ ]:
timestamps_test = pd.date_range(test_df_viz.iloc[0]["time"], periods=CONTEXT_LENGTH, freq=meta.freq)
ts_start, ts_end = timestamps_test[0], timestamps_test[-1]


def get_preds_df_filter(forecasts: np.array):
    preds = ListDataset(
        [
            {
                FieldName.TARGET: forecasts,
                FieldName.START: ts_start.strftime("%Y-%m-%d %H:%M:%S"),
            }
        ],
        freq=meta.freq,
        one_dim_target=False,
    )
    preds_df = multivar_df(next(iter(preds)))
    preds_df_filter = preds_df.loc[:, ["time"] + selected_cols]
    preds_df_filter = pd.melt(preds_df_filter, id_vars=["time"], value_vars=selected_cols)
    preds_df_filter.rename(columns={"variable": "covariate"}, inplace=True)
    return preds_df_filter


lstnet_preds_df_filter = get_preds_df_filter(np.transpose(np.array(predicted_lstnet[0])))
deepar_preds_df_filter = get_preds_df_filter(np.array([x.samples[0] for x in predicted_deepar]))

Set the time series indexes that we want to plot

In [ ]:
selected_ts = [
    "ts_98",
    "ts_100",
    "ts_116",
    "ts_137",
    "ts_147",
    "ts_156",
    "ts_184",
    "ts_216",
    "ts_245",
    "ts_267",
]

In [ ]:
selection = alt.selection_multi(fields=["covariate"], bind="legend", nearest=True)

train_plot = (
    alt.Chart(
        train_df_viz.loc[train_df_viz["covariate"].isin(selected_ts)],
        title="Input data",
    )
    .mark_line()
    .encode(
        alt.X("time:T", axis=alt.Axis(title="Time", format="%e %b, %y")),
        alt.Y(
            "value:Q",
            axis=alt.Axis(title="Electricity consumption (kW)"),
            scale=alt.Scale(domain=[0, 3000]),
        ),
        alt.Color("covariate:N"),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .add_selection(selection)
)

gt_plot = (
    alt.Chart(
        test_df_viz.loc[test_df_viz["covariate"].isin(selected_ts)],
        title="Expected output data (ground truth)",
    )
    .mark_line()
    .encode(
        alt.X(
            "time:T",
            axis=alt.Axis(title="Time", format="%e %b, %y"),
            scale=alt.Scale(domain=[ts_start, ts_end]),
        ),
        alt.Y(
            "value:Q",
            axis=alt.Axis(title="Electricity consumption (kW)"),
            scale=alt.Scale(domain=[0, 3500]),
        ),
        alt.Color("covariate:N"),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .add_selection(selection)
)

lstnet_preds_plot = (
    alt.Chart(
        lstnet_preds_df_filter.loc[lstnet_preds_df_filter["covariate"].isin(selected_ts)],
        title="LSTNet predictions",
    )
    .mark_line()
    .encode(
        alt.X(
            "time:T",
            axis=alt.Axis(title="Time", format="%e %b, %y"),
            scale=alt.Scale(domain=[ts_start, ts_end]),
        ),
        alt.Y(
            "value:Q",
            axis=alt.Axis(title="Electricity consumption (kW)"),
            scale=alt.Scale(domain=[0, 3500]),
        ),
        alt.Color("covariate:N"),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .add_selection(selection)
)


deepar_preds_plot = (
    alt.Chart(
        deepar_preds_df_filter.loc[deepar_preds_df_filter["covariate"].isin(selected_ts)],
        title="DeepAR predictions",
    )
    .mark_line()
    .encode(
        alt.X(
            "time:T",
            axis=alt.Axis(title="Time", format="%e %b, %y"),
            scale=alt.Scale(domain=[ts_start, ts_end]),
        ),
        alt.Y(
            "value:Q",
            axis=alt.Axis(title="Electricity consumption (kW)"),
            scale=alt.Scale(domain=[0, 3500]),
        ),
        alt.Color("covariate:N"),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .add_selection(selection)
)

(train_plot | gt_plot) & (lstnet_preds_plot) & deepar_preds_plot

Another way to visualize the results

In [ ]:
train_and_test_df_viz = pd.concat([train_df_viz, test_df_viz])
train_and_test_df_viz["Type"] = "Ground truth"
lstnet_preds_df_filter["Type"] = "LSTNet"
deepar_preds_df_filter["Type"] = "DeepAR"
merge_preds_df_filter = pd.concat(
    [
        train_and_test_df_viz,
        lstnet_preds_df_filter,
        deepar_preds_df_filter,
    ]
)

In [ ]:
selected_ts = ["ts_100"]
selection = alt.selection_multi(fields=["Type"], bind="legend", nearest=True)
merge_preds_plot = (
    alt.Chart(
        merge_preds_df_filter.loc[merge_preds_df_filter["covariate"].isin(selected_ts)],
        title=f'All predictions for "{selected_ts[0]}"',
    )
    .mark_line()
    .encode(
        alt.X("time:T", axis=alt.Axis(title="Time", format="%e %b, %y")),
        alt.Y(
            "value:Q",
            axis=alt.Axis(title="Electricity consumption (kW)"),
            scale=alt.Scale(domain=[0, 3500]),
        ),
        alt.Color(
            "Type",
            scale=alt.Scale(
                domain=["Ground truth", "LSTNet", "DeepAR"],
                range=["black", "brown", "blue", "green"],
            ),
        ),
        strokeDash=alt.condition(
            alt.datum.Type != "Ground truth",
            alt.value([5, 3]),
            alt.value([0]),
        ),
        strokeWidth=alt.condition(alt.datum.Type != "Ground truth", alt.value(1.5), alt.value(2)),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    )
    .add_selection(selection)
    .properties(width=800, height=400)
)
rules = (
    alt.Chart(pd.DataFrame({"Date": [ts_start.strftime("%Y-%m-%d %H:%M:%S")], "color": ["red"]}))
    .mark_rule()
    .encode(
        x="Date:T",
        color=alt.Color("color:N", scale=None, legend=alt.Legend(title="Type")),
        strokeWidth=alt.value(2),
    )
)

merge_preds_plot + rules

The training and test data (ground truth) are shown as the black solid line (seperated by red vertical line) in the plot. The predictions from different forecasting algorithms are shown as dash lines. The closer the dash line comes to the black solid line, the more accurate the predictions are.

## Clean Up the Resources

After you are done using this notebook, delete the model and the endpoint to avoid any incurring charges.

**Caution**: You need to manually delete resources that you may have created while running the notebook, such as Amazon S3 buckets for model artifacts, training datasets, processing artifacts, and Amazon CloudWatch log groups.

In [ ]:
predictor.delete_model()  # LSTNet without HPO
predictor.delete_endpoint()  # LSTNet without HPO

tuning_predictor.delete_model()  # LSTNet with HPO
tuning_predictor.delete_endpoint()  # LSTNet with HPO

deepar_predictor.delete_model()  # DeepAR without HPO
deepar_predictor.delete_endpoint()  # DeepAR without HPO

deepar_predictor_hpo.delete_model()  # DeepAR with HPO
deepar_predictor_hpo.delete_endpoint()  # DeepAR with HPO

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-1/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-east-2/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/us-west-1/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ca-central-1/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/sa-east-1/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-1/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-2/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-west-3/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-central-1/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/eu-north-1/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-1/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-southeast-2/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-1/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-northeast-2/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://h75twx4l60.execute-api.us-west-2.amazonaws.com/sagemaker-nb/ap-south-1/introduction_to_applying_machine_learning|deep_demand_forecasting|deep_demand_forecast.ipynb)
